# 1. Setup

In [1]:
import pandas as pd

In [2]:
# Đọc dữ liệu đã được transform từ file CSV
df = pd.read_csv('D:\\HTP\\Project-personal\\PriceCar\\dataset\\processed\\car.csv')

# 2. create_dim_table

In [3]:
def create_dim_table(df, id_column_name, *column_names):
    # Lấy tất cả các cột truyền vào ngoài id_column_name
    dim_table = df[list(column_names)].drop_duplicates().reset_index(drop=True)
    
    # Tạo id cho các bảng Dim
    dim_table[id_column_name] = dim_table.index + 1  # Tạo id cho bảng Dim
    return dim_table[[id_column_name] + list(column_names)]


# Tạo các bảng Dim
dim_car_model = create_dim_table(df, 'car_model_id', 'car_model')
dim_car_type = create_dim_table(df, 'car_type_id', 'car_type')
dim_color = create_dim_table(df, 'color_id', 'out_color')
dim_poster = create_dim_table(df, 'poster_id', 'poster_name', 'poster_add', 'poster_tel')
dim_car_condition = create_dim_table(df, 'condition_id', 'new_old')

# 3. create_fact_table

In [4]:
def create_fact_table(df, dim_car_model, dim_car_type, dim_color, dim_poster, dim_car_condition):
    # Tạo bảng Fact từ các cột số liệu và các cột cần thiết
    fact_car_sale = df[['km', 'car_type', 'out_color', 'in_color', 'door_num', 'seat_num', 
                        'new_old', 'car_year', 'title', 'price_words', 'price_num', 'area']].copy()

    # Thêm các cột khóa ngoại từ các bảng Dim
    fact_car_sale['car_model_id'] = df['car_model'].map(dict(zip(dim_car_model['car_model'], dim_car_model['car_model_id'])))
    fact_car_sale['car_type_id'] = df['car_type'].map(dict(zip(dim_car_type['car_type'], dim_car_type['car_type_id'])))
    fact_car_sale['color_id'] = df['out_color'].map(dict(zip(dim_color['out_color'], dim_color['color_id'])))
    fact_car_sale['poster_id'] = df['poster_name'].map(dict(zip(dim_poster['poster_name'], dim_poster['poster_id'])))
    fact_car_sale['condition_id'] = df['new_old'].map(dict(zip(dim_car_condition['new_old'], dim_car_condition['condition_id'])))

    # Định nghĩa thứ tự các cột
    fact_car_sale = fact_car_sale[['car_model_id', 'car_type_id', 'color_id', 'poster_id', 'condition_id', 
                                   'km', 'door_num', 'seat_num', 'car_year', 'price_num', 
                                   'price_words', 'area']]
    return fact_car_sale


# Tạo bảng Fact_Car_Sale
fact_car_sale = create_fact_table(df, dim_car_model, dim_car_type, dim_color, dim_poster, dim_car_condition)
fact_car_sale.head()

,car_model_id,car_type_id,color_id,poster_id,condition_id,km,door_num,seat_num,car_year,price_num,price_words,area
0,1,1,1,1,1,0,5,7,2007,630000000,630 Triệu,TP HCM
1,1,1,2,15376,1,0,5,7,2007,580000000,580 Triệu,Hà Nội
2,1,1,2,3,1,0,5,7,2006,465000000,465 Triệu,TP HCM
3,1,1,1,4,2,0,5,7,2016,4000000000,4 Tỷ 665 Triệu,TP HCM
4,1,1,3,5,1,9600,5,7,2015,3000000000,3 Tỷ 400 Triệu,TP HCM


# 4. save

In [5]:
def save_to_csv(table, file_path):
    table.to_csv(file_path, index=False)
    print(f"Đã lưu {file_path} thành công.")

# Lưu các bảng vào file CSV
save_to_csv(dim_car_model, 'D:\\HTP\\Project-personal\\PriceCar\\dataset\\processed\\Dim_Car_Model.csv')
save_to_csv(dim_car_type, 'D:\\HTP\\Project-personal\\PriceCar\\dataset\\processed\\Dim_Car_Type.csv')
save_to_csv(dim_color, 'D:\\HTP\\Project-personal\\PriceCar\\dataset\\processed\\Dim_Color.csv')
save_to_csv(dim_poster, 'D:\\HTP\\Project-personal\\PriceCar\\dataset\\processed\\Dim_Poster.csv')
save_to_csv(dim_car_condition, 'D:\\HTP\\Project-personal\\PriceCar\\dataset\\processed\\Dim_Car_Condition.csv')
save_to_csv(fact_car_sale, 'D:\\HTP\\Project-personal\\PriceCar\\dataset\\processed\\Fact_Car_Sale.csv')

Đã lưu D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Car_Model.csv thành công.
Đã lưu D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Car_Type.csv thành công.
Đã lưu D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Color.csv thành công.
Đã lưu D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Poster.csv thành công.
Đã lưu D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Car_Condition.csv thành công.
Đã lưu D:\HTP\Project-personal\PriceCar\dataset\processed\Fact_Car_Sale.csv thành công.
